In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.2.0/spark-3.2.0-bin-hadoop3.2.tgz
!tar xf spark-3.2.0-bin-hadoop3.2.tgz
!pip install -q findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "spark-3.2.0-bin-hadoop3.2"

In [ ]:
import findspark
findspark.init()

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
sc = spark.sparkContext
sc

<SparkContext master=local[*] appName=pyspark-shell>

# PART A

In [ ]:
# reading the wildlife trade file
trade_data = spark.read.csv('/content/wildlife_trade.csv',header=True)

In [ ]:
# schema of the trade data frame
trade_data.schema

StructType(List(StructField(Year,StringType,true),StructField(App.,StringType,true),StructField(Taxon,StringType,true),StructField(Class,StringType,true),StructField(Order,StringType,true),StructField(Family,StringType,true),StructField(Genus,StringType,true),StructField(Term,StringType,true)))

In [ ]:
trade_data.show()

+----+----+--------------------+-----------+----------------+-------------+----------+---------+
|Year|App.|               Taxon|      Class|           Order|       Family|     Genus|     Term|
+----+----+--------------------+-----------+----------------+-------------+----------+---------+
|2016|   I|      Aquila heliaca|       Aves|   Falconiformes| Accipitridae|    Aquila|   bodies|
|2016|   I|      Aquila heliaca|       Aves|   Falconiformes| Accipitridae|    Aquila|   bodies|
|2016|   I|Haliaeetus albicilla|       Aves|   Falconiformes| Accipitridae|Haliaeetus| feathers|
|2016|   I|Haliaeetus albicilla|       Aves|   Falconiformes| Accipitridae|Haliaeetus|specimens|
|2016|   I|Haliaeetus albicilla|       Aves|   Falconiformes| Accipitridae|Haliaeetus|specimens|
|2016|   I|Haliaeetus albicilla|       Aves|   Falconiformes| Accipitridae|Haliaeetus|   bodies|
|2016|   I|      Harpia harpyja|       Aves|   Falconiformes| Accipitridae|    Harpia| feathers|
|2016|   I|      Harpia harpyj

In [ ]:
trade_data.createOrReplaceTempView('Trade')

# question 1 - what is the most frequent Class of animal traded

# Selecting the most frequent class using SQL statements by grouping by class and ordering by count
animal_freq_trades = spark.sql('SELECT Class FROM Trade WHERE Class is not null GROUP BY Class ORDER BY Count("App.") DESC LIMIT 1')
animal_freq_trades.show()

+--------+
|   Class|
+--------+
|Reptilia|
+--------+



In [ ]:
# question 2 - List all the items (Term) traded that are associated with Mammals

# Selecting the distinct Term where class is mammalia
mammals_items = spark.sql('SELECT DISTINCT Term FROM Trade WHERE Class == "Mammalia"')
mammals_items.show()

+--------------------+
|                Term|
+--------------------+
|                gall|
|                meat|
| fur product (small)|
|              fibres|
|              scales|
|         bone pieces|
|            garments|
|            medicine|
|              bodies|
|            carvings|
|               horns|
|leather products ...|
|               claws|
|              skulls|
|             extract|
|               tails|
|                hair|
|           specimens|
|         skin pieces|
|         horn pieces|
+--------------------+
only showing top 20 rows



In [ ]:
# question 3 - List all CITES Appendix II species

from pyspark.sql import Row

# converting the data frame to rdd and filtering the rows where Appendix is II
app2_list = spark.createDataFrame(trade_data.rdd.filter(lambda x : x['App.']=='II').map(lambda x: Row(App=x['App.'], Taxon=x['Taxon'], Class=x['Class'], Order=x['Order'], Family=x['Family'], Genus=x['Genus']))).distinct()
app2_list.show()

+---+--------------------+--------+--------------+------------+-------------+
|App|               Taxon|   Class|         Order|      Family|        Genus|
+---+--------------------+--------+--------------+------------+-------------+
| II| Acropora natalensis|Anthozoa|  Scleractinia| Acroporidae|     Acropora|
| II|  Acropora subglabra|Anthozoa|  Scleractinia| Acroporidae|     Acropora|
| II| Montipora millepora|Anthozoa|  Scleractinia| Acroporidae|    Montipora|
| II|  Crocodylus porosus|Reptilia|    Crocodylia|Crocodylidae|   Crocodylus|
| II|     Grampus griseus|Mammalia|       Cetacea| Delphinidae|      Grampus|
| II|    Falco sparverius|    Aves| Falconiformes|  Falconidae|        Falco|
| II|      Uroplatus spp.|Reptilia|        Sauria|  Gekkonidae|    Uroplatus|
| II|        Aloe confusa|    null|      Liliales|   Liliaceae|         Aloe|
| II|        Aloe falcata|    null|      Liliales|   Liliaceae|         Aloe|
| II|       Barkeria spp.|    null|    Orchidales| Orchidaceae| 

In [ ]:
# question 4 - What is the most common animal (Taxon) traded in 2017?

# Selecting most common taxon from the year 2017
common_taxon = spark.sql("SELECT Taxon FROM Trade WHERE Year == 2017 GROUP BY Taxon ORDER BY COUNT(Taxon) DESC LIMIT 1")
common_taxon.show()

+--------------------+
|               Taxon|
+--------------------+
|Alligator mississ...|
+--------------------+



In [ ]:
# question 5 - List all the Classes of animals where the following items are traded a) teeth b) live c) carvings

# Selecting distinct class where the given items are traded
list_classes = spark.sql("SELECT DISTINCT Class FROM Trade WHERE Term in ('teeth','live','carvings')")
list_classes.show()

+--------------+
|         Class|
+--------------+
|          Aves|
|      Bivalvia|
|      Amphibia|
|       Insecta|
|      Mammalia|
|Elasmobranchii|
|     Arachnida|
|          null|
|    Gastropoda|
|      Reptilia|
|   Actinopteri|
|      Anthozoa|
|     Dipneusti|
|  Hirudinoidea|
|      Hydrozoa|
+--------------+

